# **OpenMP**

OpenMP (Open Multi-Processing) is an application programming interface (API) that supports multi-platform shared-memory multiprocessing programming in C, C++, and Fortran. It consists of a set of compiler ```#pragmas``` that control how the program works. The pragmas are designed so that even if the compiler does not support them, the program will still yield correct behavior, but without any parallelism.


## **1 - Execution Model**

The OpenMP API uses the fork-join model of parallel execution. Multiple threads of execution perform tasks deﬁned implicitly or explicitly by OpenMP directives. The OpenMP API is intended to support programs that will execute correctly both as parallel programs (multiple threads of execution and a full OpenMP support library) and as sequential programs (directives ignored and a simple OpenMP stubs library). However, a conforming OpenMP program may execute correctly as a parallel program but not as a sequential program, or may produce diﬀerent results when executed as a parallel program compared to when it is executed as a sequential program. Further, using diﬀerent numbers of threads may result in diﬀerent numeric results because of changes in the association of numeric operations. For example, a serial addition reduction may have a diﬀerent pattern of addition associations than a parallel reduction. These diﬀerent associations may change the results of ﬂoating-point addition.

An OpenMP program begins as a single thread of execution, called an initial thread (also known as Master thread), as depicted in Figure below. An initial thread executes sequentially until it reaches a parallel directive that tells the compiler to generate parallel code (fork). At this point, worker threads are created to execute the parallel region. At the end of a parallel region, there is an implicit synchronization to join all the threads (join). From this point on, only the master thread executes. This process repeats for every new parallel region in the code.

![img.gif](../imgs/fork_join1.gif)

## **2 - OpenMP syntax**

All OpenMP constructs in C and C++ are indicated with a ```#pragma omp``` followed by parameters, ending in a newline. The pragma usually applies only into the statement immediately following it, except for the ```barrier``` and ```flush``` commands, which do not have associated statements.


### **2.1 - Directive Format**

OpenMP directives are speciﬁed with a directive-speciﬁcation. A directive-speciﬁcation consists of the directive-speciﬁer and any clauses that may optionally be associated with the OpenMP directive. An OpenMP directive may be specified as a pragma directive:
  
```
#pragma omp directive-specification new-line
```

or a pragma operator:

```
_Pragma("omp directive-specification")
```

### **2.2 - Clause Format**

OpenMP clauses are speciﬁed as part of a directive-speciﬁcation. Clauses are optional and, thus, may be omitted from a directive-speciﬁcation unless otherwise speciﬁed. The order in which clauses appear on directives is not signiﬁcant unless otherwise speciﬁed.

```#pragma omp directive-specification clause-name[(clause-argument-speciﬁcation [; clause-argument-speciﬁcation [;...]])]```


---

## **3 - Exploiting Parallelism**

This section discuss constructs for generating and controlling parallelism.


### **3.1 - The Parallel construct**

The parallel construct starts a parallel block. It creates a *team* of N threads (where N is determined at runtime, usually from the number of CPU cores), all of which execute the next statement (or the next block). After the statement, the threads join back into one.

```
int main(int argc, char **argv){
    \* sequential region *\

    #pragma omp parallel
    {
        \* parallel region *\
    }

    \* sequential region *\

    return 0;
}
```

Within a ```parallel``` region, thread numbers uniquely identify each thread (from 0 to the number of created threads - 1). A thread can obtain its own thread number by calling the function ```omp_get_thread_num()```. A thread can also get the total number of threads that are running the ```parallel``` region through the function ```omp_get_num_threads()```.

If a thread in a ```parallel``` region encounters another ```#pragma omp parallel``` directive, it creates a new team of threads. 

An implicit barrier occurs at the end of a ```parallel``` region to join all threads. After the end of such region, only the master thread of the team resumes execution.




#### Exercise 1: Parallelizing the first code

For this exercise, you must apply the ```#pragma omp parallel```to parallelize the code [hello_seq.c](../src/introduction/hello/hello_seq.c).
1. First, compile and run the sequential code to observe that only one ```Hello World``` is printed in the screen.

In [1]:
!cd ../src/introduction/hello && gcc hello_seq.c -o hello_seq && ./hello_seq

Hello World


2. Open the code [hello_omp.c](../src/introduction/hello/hello_omp.c), include the libgomp library, and apply the parallel directive so that multiple ```Hello World``` are printed out. Once you are done, save the file.
3. To compile an OpenMP code with GCC compiler, you must include the ```-fopenmp``` flag to the command line: ```gcc file.c -o a.out -fopenmp```. 
4. Play the next cell to compile and run the binary.

In [2]:
!cd ../src/introduction/hello && gcc hello_omp.c -o hello_omp -fopenmp && ./hello_omp

Hello World



### **3.1.1 - Playing with the number of threads**

By default, the number of threads that are created to execute each parallel region matches the number of cores in the architecture. However, the user can modify this number via environment variable, function in the source code, or via clause.
1. **Environment Variable**: In the Linux terminal, you can type the following command to define the number of threads equal to 4:
    ```
    export OMP_NUM_THREADS=4
    ```
2. **Function**: Inside the code, the user can use the following function to define the number of threads equal to 4:
    ```
    omp_set_num_threads(4);
    ```
3. **Clause**: Along with the parallel construct, the user can use the following clause to define the number of threads equal to 4:
    ```
    #pragma omp parallel num_threads(4)
    ```

#### Exercise 2: Changing the number of threads

For this exercise, you must apply the previous knowledge acquired to change the number of running threads in the parallel region for the code [hello_omp.c](src/introduction/hello/hello_omp.c).

##### 2.A: Through Environment Variable
To use the environment variable, you can type it directly into the terminal where the application is running. Try with different number of threads.

In [3]:
!cd ../src/introduction/hello/ && export OMP_NUM_THREADS=4 && ./hello_omp

Hello World


##### 2.B: Through Function
To use the function, you need to write it before the parallel constructor. Therefore, edit the code [here](src/introduction/hello/hello_omp.c). Once you have finished, save it and play the next cell.

In [ ]:
!cd ../src/introduction/hello/ && gcc hello_omp.c -o hello_omp -fopenmp && ./hello_omp

##### 2.C: Through Clause
To use the clause, you just need to write it in the end of the ```#pragma omp parallel``` directive. Therefore, edit the code [here](src/introduction/hello/hello_omp.c). Once you have finished, save it and play the next cell.


In [ ]:
!cd ../src/introduction/hello/ && gcc hello_omp.c -o hello_omp -fopenmp && ./hello_omp

##### 2.D: Try different approaches

After trying the different ways of defining the number of threads, take a time to do the following (For all cases below, you can run the same cell after each edit):
1. Create more than one parallel region in the source code and identify each parallel region with a different printf. 
2. What happens when the number of threads is defined via environment variable?
3. What happens when the number of threads is defined via function? Try using this function with different number of threads right before each parallel region.
4. What happens when the number of threads is defined via clause? Try using different values for each parallel region.
5. Is there a precedence when defining the number of threads?

In [ ]:
!cd ../src/introduction/hello/ && gcc hello_omp.c -o hello_omp -fopenmp && ./hello_omp

#### Exercise 3: What is wrong with the code?

In this exercise, you will run an application that performs the addVectors operation in parallel. This application is parallelized with only the ```#pragma omp parallel``` directive. You can find the source code [here](src/introduction/addVectors/addVectors.c). It performs the sum of 8 elements of array C = A + B; To run, play the next cell:


In [ ]:
!cd ../src/introduction/addVectors/ && gcc addVectors.c -o addVectors -fopenmp && ./addVectors

1. What is the output?
2. Why all threads are computing the same indices of each array?
3. Take a time to understand what is happening in this exercise. Also, remember that the ```#pragma omp parallel``` directive only creates a team of threads. That is, it does not divide the workload (iterations) among the threads. 
4. We will cover the worksharing constructor in the next chapter.

### **3.1.2 - Clauses**

#### **3.1.2.1 - private(list)**

This clause declares the scope of the data variables in ```list``` to be private to each thread during the parallel region. Data variables in list are separated by commas, as exemplified below for variables ```a``` and ```b``` .

```
int a, b, c;
#pragma omp parallel private(a, b)
{
    a = omp_get_thread_num();
    b = srand();
    printf("thread id %d generated a random value %d\n", a, b);
}

```


#### **3.1.2.2 - shared(list)**

When this clause is employed, the scope of the comma-separated data variables in list are defined to be shared across all threads. By default, all variables are shared among the threads. In the example below, the array ```A``` is declared to be shared.

```
int A[N];
#pragma omp parallel shared(A)
{
    int sum_per_thread = 0;
    for(int i = 0; i < N; i++){
        sum_per_thread += A[i] * omp_get_thread_num();
    }    
}
```

#### **3.1.2.3 - firstprivate(list)**

When this clause is applied to a parallel region, the scope of the data variables in ```list``` to be private to each thread. Different from the ```private``` clause, each new private variable is initialized with the value of the original variable as if there was an implied declaration within the statement block. Also, data variables in list are separated by commas. In the example below, the printed values for ```a``` and ```b``` should be 100 and 10, respectively.

```
int a = 100, b = 10, c;
#pragma omp parallel private(c) firstprivate(a, b)
{
    printf("thread id %d with values a (%d), b(%d) and c(%d)\n", omp_get_thread_num(), a, b, c);
}

```

#### **3.1.2.4 - num_threads(int n)**

When this clause is employed, the value of ```n``` is an integer expression that specifies the number of threads to use for the parallel region. If dynamic adjustment of the number of threads is also enabled, then ```n``` specifies the maximum number of threads to be used. In the example below, 16 threads should be created and each one should print its id along with the number 16.

```
int nThreads = 16;
int idThread, totalThreads;
#pragma omp parallel num_threads(nThreads) private(idThread) shared(totalThreads)
{
    idThread = omp_get_thread_num();
    totalThreads = omp_get_num_threads();
    printf("I am thread %d from a total of %d threads\n", idThread, totalThreads);       
}
```


#### **3.1.2.5 - reduction(operator: list)**

This clause performs a reduction on all scalar variables in ```list``` using the specified ```operator```. Reduction variables in list are separated by commas. When it is employed, a private copy of each variable in list is created for each thread. At the end of the statement block, the final values of all private copies of the reduction variable are combined in a manner appropriate to the operator, and the result is placed back in the original value of the shared reduction variable. For example, when the max operator is specified, the original reduction variable value combines with the final values of the private copies by using the following expression:

```
    original_reduction_variable = original_reduction_variable < private_copy ?
    private_copy : original_reduction_variable;
```

When the programming language is C or C++, the following operators can be used.
 
<table><tr><th > Identifier	<th><th>   Initializer <th><th>   Combiner <tr><tr>
<tr><td> +	                <td><td>   omp_priv = 0	            <td><td>    omp_out += omp_in <tr><tr>
<tr><td> -	                <td><td>   omp_priv = 0	            <td><td>    omp_out += omp_in <tr><tr>
<tr><td> *	                <td><td>   omp_priv = 1	            <td><td>    omp_out *= omp_in <tr><tr>
<tr><td> &	                <td><td>   omp_priv = ~ 0	        <td><td>    omp_out &= omp_in <tr><tr>
<tr><td> |	                <td><td>   omp_priv = 0	            <td><td>    omp_out |= omp_in <tr><tr>
<tr><td> ^	                <td><td>   omp_priv = 0	            <td><td>    omp_out ^= omp_in <tr><tr>
<tr><td> &&	            <td><td>   omp_priv = 1	                <td><td>    omp_out = omp_in && omp_out <tr><tr>
<tr><td> ||	            <td><td>   omp_priv = 0	                <td><td>    omp_out = omp_in || omp_out <tr><tr>
<tr><td> max	            <td><td>   omp_priv = Least representable number in the reduction list item type	<td><td>    omp_out = omp_in > omp_out ? omp_in : omp_out <tr><tr>
<tr><td> min	            <td><td>   omp_priv = Largest representable number in the reduction list item type	<td><td>    omp_out = omp_in < omp_out ? omp_in : omp_out <tr><tr><table>

The following example shows the reduction ```+``` operation over the variable ```sum```.

```
int sum=0;
#pragma omp parallel reduction(+:sum)
{
    int idThread = omp_get_thread_num();
    sum += idThread;    
}
printf("The sum of all thread ids is: %d\n", sum);       
```

### **3.2 - Work-sharing constructs**

A work-sharing construct delegates the execution of the corresponding region among the threads within its designated thread team. Threads execute portions of the parallel region. 

### **3.2.1 - \#pragma omp for**

The ```omp for``` directive informs the compiler to distribute loop iterations within the team of threads that reaches this constructor. The syntax is ```#pragma omp for clauses```, where the clauses are optional and will be discussed next.

A simple example can be seen below, where the number of iterations ```N``` will be divided among the threads that are created in the parallel directive. In this scenario, each created thread will be responsible for executing a different set of iterations.

```
#pragma omp parallel
{
    #pragma omp for
    for(int i = 0; i < N; i++){
        \* loop operations in parallel *\
    }
}

```

The ```pragma omp for``` directive can be combined with the ```pragma omp parallel```. In this scenario, the following code has the very same effect as the previous one.

```
#pragma omp parallel for
for(int i = 0; i < N; i++){
    \* loop operations in parallel *\
}

```

#### Exercise 4: Applying the Work-sharing constructor for the addVectors code

In this exercise, you will modify the [addVectors](src/introduction/addVectors/addVectors.c) application so that the iterations of the loop are distributed among the threads in the parallel region through the ```schedule``` clause.
After editing and saving the code, play the following cell.

In [ ]:
!cd ../src/introduction/addVectors/ && gcc addVectors.c -o addVectors -fopenmp && ./addVectors

1. What is the output? 
2. Are the threads computing different indices of each array?
3. Change the number of threads that are created in the parallel region and run the cell below again. What happens?

In [ ]:
!cd ../src/introduction/addVectors/ && gcc addVectors.c -o addVectors -fopenmp && ./addVectors

#### **3.2.1.1 - schedule(type) clause**

This clause specifies how iterations of the for loop are divided among available threads. Acceptable values for ```type``` are:

- **static**: Iterations of a loop are divided into chunks of size *ceiling(number_of_iterations/number_of_threads)*. Each thread is assigned a separate chunk. This scheduling policy is also known as block scheduling.

- **static,n**: Iterations of a loop are divided into chunks of size *n*. Each chunk is assigned to a thread in round-robin fashion. *n* must be an integral assignment expression of value 1 or greater.

- **dynamic**: Iterations of a loop are divided into chunks of size *ceiling(number_of_iterations/number_of_threads)*.
Chunks are dynamically assigned to active threads on a "first-come, first-do" basis until all work has been assigned.

- **dynamic,n**: As above, except chunks are set to size *n*. *n* must be an integral assignment expression of value 1 or greater.

- **guided**: Chunks are made progressively smaller until the default minimum chunk size is reached. The first chunk is of size *ceiling(number_of_iterations/number_of_threads)*. Remaining chunks are of size *ceiling(number_of_iterations_left/number_of_threads)*. The minimum chunk size is 1. Chunks are assigned to active threads on a "first-come, first-do" basis until all work has been assigned.

- **guided,n**: As above, except the minimum chunk size is set to *n*; *n* must be an integral assignment expression of value 1 or greater.

- **runtime**: Scheduling policy is determined at run time. Use the OMP_SCHEDULE environment variable to set the scheduling type and chunk size.

- **auto**: the scheduling is delegated to the compiler and runtime system. The compiler and runtime system can choose any possible mapping of iterations to threads (including all possible valid schedules) and these may be different in different loops.


![scheduling.png](../imgs/scheduling.png)

source of image: https://www.micc.unifi.it/bertini/download/parallel/2016-2017/9_shared_memory_openmp_directives.pdf

#### Exercise 5: Playing with different scheduler types.

In this exercise, you will modify the [addVectors](src/introduction/addVectors/addVectors.c) application so that the iterations of the loop are distributed among the threads in the parallel region considering the types defined above. For that, try the following:
1. Use the clause ```schedule(static,3)```. Play the next cell. What is the output?

In [ ]:
!cd ../src/introduction/addVectors/ && gcc addVectors.c -o addVectors -fopenmp && ./addVectors

2. Use the clause ```schedule(dynamic,2)```. Play the next cell. What is the output?

In [ ]:
!cd ../src/introduction/addVectors/ && gcc addVectors.c -o addVectors -fopenmp && ./addVectors

3. Use the clause ```schedule(guided,2)```. Play the next cell. What is the output?

In [ ]:
!cd ../src/introduction/addVectors/ && gcc addVectors.c -o addVectors -fopenmp && ./addVectors

4. Repeat the process above changing the type and the chunksize. Also, change the value of ```N``` so the behavior of each schedule type can be better seen.

In [ ]:
!cd ../src/introduction/addVectors/ && gcc addVectors.c -o addVectors -fopenmp && ./addVectors

#### **3.2.1.2 - nowait clause**

This clause can be employed to avoid the implied barrier at the end of the ```for``` directive. This is useful when there are multiple independent work-sharing sections or iterative loops within a given parallel region. Only one nowait clause can appear on a given for directive. The example below describes the usage on a parallel region with two loops.


```
#pragma omp parallel
{
    #pragma omp for nowait
    for(int i = 0; i < N; i++){
        \* loop operations in parallel *\
    }

    #pragma omp for nowait
    for(int i = 0; i < N; i++){
        \* loop operations in parallel *\
    }
}

```

#### **3.2.1.3 - collapse(n) clause**

When this clause is employed, it allows you to parallelize *n* multiple loops in a nest without introducing nested parallelism.
For that, the loops must form a rectangular iteration space and the bounds and stride of each loop must be invariant over all the loops. If the loop indices are of different size, the index with the largest size will be used for the collapsed loop.
The loops must be perfectly nested; that is, there is no intervening code nor any OpenMP pragma between the loops which are collapsed. 
The associated do-loops must be structured blocks. Their execution must not be terminated by a **break** statement.
If multiple loops are associated to the loop construct, only an iteration of the innermost associated loop may be curtailed by a continue statement. If multiple loops are associated to the loop construct, there must be no branches to any of the loop termination statements except for the innermost associated loop.

The following example depicts the parallelization of two loops that are collapsed.

```
#pragma omp parallel for collapse(2)
for(int i = 0; i < N; i++){
    for(int j = 0; j < N; j++){
        \* loop operations in parallel *\
    }
}

```

#### **3.2.1.4 - ordered clause**

When this clause is employed, during execution of an iteration of a loop or a loop nest within a loop region, the executing thread must not execute more than one ordered region which binds to the same loop region. As a consequence, if multiple loops are associated to the loop construct by a collapse clause, the ordered construct has to be located inside all associated loops.


### **3.2.2 - \#pragma omp single**

The ```single``` construct specifies that the associated structured block is executed by only one of the threads in the team (not necessarily the master thread), in the context of its implicit task. The other threads in the team, which do not execute the block, wait at an implicit barrier at the end of the single construct unless a nowait clause is specified. In the following example, only one thread will execute the printf operation.

```
#pragma omp parallel
{
    #pragma omp single
    {
        printf("Total number of threads: %d\n", omp_get_num_threads());
    }
    \* operations done in parallel *\

}

```

### **3.2.3 - \#pragma omp sections**

When this work-sharing constructor is employed, a set of structured blocks are distributed among and executed by the threads in a team. Each structured block is executed once by one of the threads in the team in the context of its implicit task. The following example depicts the execution of two operations in parallel by two threads (each thread executes a different section).


```
#pragma omp parallel
{
    #pragma omp sections
    { 
        #pragma omp section 
        {
            printf("Thread %d is running the first structured block\n", omp_get_thread_num());
        } 
        #pragma omp section 
        {
            printf("Thread %d is running the second structured block\n", omp_get_thread_num());
        } 
   }
}
```

When this work-sharing is employed, the parallelization scheme is very similar to the one applied by POSIX Threads. In this scenario, the programmer is responsible for dividing the workload among threads in sections.

#### Exercise 6: Applying sections to the addVectors code

In this exercise, you will modify the [addVectors](src/introduction/addVectors/addVectors.c) application so that the parallelization scheme employed is ```sections```. In this scenario, consider that **two** threads will be created (and hence, two sections) and the iterations of the loop are distributed equaly among the threads.
After editing and saving the code, play the following cell.
Take a time to change the number of threads that are created. What happens with the execution? Are all the threads executing the parallel region (how can you check this?)

In [ ]:
!cd ../src/introduction/addVectors/ && gcc addVectors_sections.c -o addVectors_sections -fopenmp && ./addVectors_sections

## **3.4 Memory Model and Data-Synchronization**

In any multithreaded programming environment, the definition of a memory model becomes crucial. This constitutes an agreement between the programmer and the environment, wherein the system operates under the assumption of rule adherence. When these rules are upheld, the system aligns its behavior with the programmer's expectations.

The OpenMP memory model comprises two fundamental components: a collective global memory accessible to all threads, and a temporary memory for each thread. Every read and write operation pertains to this temporary perspective, which may not necessarily align with the state of the global memory. 

Full consistency is assured solely subsequent to a *flush* operation, which ensures synchronization between the written content in one's private memory view and its visibility in the global memory, as well as making data flushed by others available for reading in one's private view. 

OpenMP inherently carries out a flush procedure upon entering or exiting a parallel region, as well as within the critical section.

The concept of a "temporary view" is merely an abstract depiction of how the system is assured to function. This does not pertain to any specific hardware or software component. In practical terms, the notion of a "temporary view" encompasses various elements, including (but not limited to):
1. The extent of compiler optimizations applied.
2. The operational mechanisms of the CPU's cache memory system.

For instance, when you modify shared data in your code, the compiler could opt to retain the present value within CPU registers until a flush is required. Alternatively, the CPU might choose to maintain the current value in the L1 cache until flushing becomes necessary.

Assume that we run the following code with **two** threads.

```
int a = 0;
#pragma omp parallel
{
    #pragma omp critical
    {
        a += 1;
    }
}
```

If thread 0 enters the critical section first, the timeline of the global memory and the temporary views of the threads are as follows:

<table><tr><th > Memory	 <th><th> Thread 0 <th><th>	Thread 1    <th><th>	Explanation <tr><tr>
<tr><td> a = ?          <td><td>	a = ?   <td><td> -          <td><td> -		<tr><tr>
<tr><td> a = ?          <td><td>	a = 0   <td><td> -          <td><td>		Thread 0 sets a = 0 <tr><tr>
<tr><td> a = 0          <td><td>	a = 0   <td><td>	a = 0   <td><td>	Threads 0 and 1 enter parallel region, both flush and everybody has a consistent view of everything <tr><tr>
<tr><td> a = 0          <td><td>	a = 0   <td><td>	a = 0   <td><td>	Thread 0 enters critical section, flush (that does nothing) <tr><tr>
<tr><td> a = ?          <td><td>	a = 1   <td><td>	a = ?   <td><td>	Thread 0 modifies a, we do not know if the change is visible in the global memory or in the temporary view of thread 1 <tr><tr>
<tr><td> a = 1          <td><td>	a = 1   <td><td>	a = ?   <td><td>	Thread 0 leaves critical section, flush, modification visible in the global memory, but thread 1 has not flushed its temporary view yet <tr><tr>
<tr><td> a = 1          <td><td>	a = 1   <td><td>	a = 1   <td><td>	Thread 1 enters critical section, flush, the latest value of a is now visible in its temporary view <tr><tr>
<tr><td> a = ?          <td><td>	a = ?   <td><td>	a = 2   <td><td>	Thread 1 modifies a, we do not know if the change is visible in the global memory or in the temporary view of thread 0 <tr><tr>
<tr><td> a = 2          <td><td>	a = ?   <td><td>	a = 2   <td><td>	Thread 1 leaves critical section, flush, modification visible in the global memory, but thread 0 has not flushed its temporary view yet <tr><tr>
<tr><td> a = 2          <td><td>	a = 2   <td><td>	a = 2   <td><td>	Threads 0 and 1 leave parallel region, flush, now also thread 0 sees the latest value <tr><tr>
<tr><td> a = 2          <td><td>	a = 2   <td><td>    -   <td><td>		Only thread 0 running, it can now read a and see what we would expect <tr><tr><table>


Each shared data element *X* can be categorized into different classes:

1. **Only one Thread**: In the parallel region, only one thread accesses *X*, either for reading or writing. This scenario necessitates no synchronization as the value of *X* remains accessible within one thread's temporary view throughout the parallel region and is exclusively flushed at the region's culmination.

2. **Read-Only Variable**: Within the parallel region, *X* remains untouched by write operations. That is, all interactions involve reading. This context eliminates the need for synchronization.

3. **Other Scenarios**: It happens when *X* is accessed for reading and writing from different threads. This needs synchronization. Accessing or modifying *X* is confined to a critical section or a synchronization construct.

### **3.4.1 - Critical Operations**

Critical regions are defined to prevent race conditions and force threads to execute a specific code segment one by one. In OpenMP, the ```#pragma omp critical``` directive is used to define this kind of region. In short, it specifies a code block that is restricted to access by only one thread at a time.

When this pragma is used, a thread waits at the beginning of a *critical* section until no other thread in the team is executing a *critical* section having the same name. Therefore, if there are *N* threads, each thread will execute this region. While a thread *I* is executing this region, other threads will have to wait before another thread will start executing the region.

The following code shows an example of using ```pragma omp critical``` to ensure that only one thread will perform the calculation of ```sum_shared``` variable at a time. 

```
int main() {
  // shared variable
    int sum_shared = 0;
    omp_set_num_threads(4);
    #pragma omp parallel
    {
        // private variable
        int sum_local = 0;
        #pragma omp for nowait
        for (int i = 0; i < 10; ++i) {
            sum_local += i;
        }
        // critical section to update the shared variable
        printf("Thread %d, local_sum = %d\n", omp_get_thread_num(), sum_local);
        #pragma omp critical
        {
            sum_shared += sum_local;
        }
    }
    printf("total sum: %d\n", sum_shared);
  return 0;
}
```

Take a time to understand the code and then, play the cell above to see the output of (critical_region)[../src/introduction/critical_region.c] code. You can change the number of threads that are created to see different behaviors of the execution.

In [1]:
!cd ../src/introduction/data/ && gcc critical_region.c -o critical_region -fopenmp && ./critical_region

Thread 2, local_sum = 13
Thread 1, local_sum = 12
Thread 0, local_sum = 3
Thread 3, local_sum = 17
total sum: 45


### **3.4.2 - Atomic Operations**

The omp atomic directive allows access of a specific memory location atomically. It ensures that race conditions are avoided through direct control of concurrent threads that might read or write to or from the particular memory location. With the omp atomic directive, you can write more efficient concurrent algorithms with fewer locks.


https://www.ibm.com/docs/en/zos/2.2.0?topic=SSLTBW_2.2.0/com.ibm.zos.v2r2.cbclx01/prag_omp_atomic.html


## **3.x - Exercises**

### **3.x.1 - Problem 1**

### **3.x.2 - Problem 2**

### **3.x.3 - Problem 3**

### **3.x.4 - Problem 4**